# ETL Process
# For this project the ETL process is quite simple
# 1.The dataset was downloaded from the website of the Namibia Statistics Agency 
# 2. The downloaded file was in SPSS format
# 3. It SPSS format was transformed into a CSV 
# 4. THe CSV then uploaded to CloudObject Storage in my IBM Watson Studio 

In [87]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'YqS2Jsc_fizSkQk91m7T3pBghp0vVXgTIvl-dtcKTR68',
    'service_id': 'iam-ServiceId-a9d9933e-9803-4a3b-b387-dc29a5129872',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token'}

configuration_name = 'os_d80919c85ea34f148938b12fae700e99_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('NHIESHH2016.csv', 'scalableds-donotdelete-pr-tmcow79opbwcdy'))
#df_data_1.take(5)


In [88]:
df.createOrReplaceTempView("df")
spark.sql("SELECT apci_groups from df").show()


+-----------+
|apci_groups|
+-----------+
|          3|
|          3|
|          5|
|          3|
|          3|
|          3|
|          2|
|          5|
|          1|
|          2|
|          2|
|          1|
|          3|
|          6|
|          4|
|          3|
|          5|
|          4|
|          6|
|          3|
+-----------+
only showing top 20 rows



In [89]:
nhiesmdl = spark.sql("SELECT sex_of_head, main_language, main_sourceincome,apci_groups from df")

In [90]:
nhiesmdl.show(5)

+-----------+-------------+-----------------+-----------+
|sex_of_head|main_language|main_sourceincome|apci_groups|
+-----------+-------------+-----------------+-----------+
|          2|            5|                1|          3|
|          2|           10|                5|          3|
|          1|            5|                1|          5|
|          2|            5|                5|          3|
|          1|           12|                1|          3|
+-----------+-------------+-----------------+-----------+
only showing top 5 rows



In [92]:
# save as csv to be retrieved later
nhiesmdl.write.format("com.databricks.spark.csv").option("header", "true").save("data/home/hiesmdl1.csv")

In [155]:
import pyspark
#sc = pyspark.SparkContext()
sql = SQLContext(sc)
#read back saved csv
dfn = (sql.read
         .format("com.databricks.spark.csv")
         .option("header", "true")
         .load("data/home/hiesmdl1.csv"))


In [156]:
dfn.show(5)

+-----------+-------------+-----------------+-----------+
|sex_of_head|main_language|main_sourceincome|apci_groups|
+-----------+-------------+-----------------+-----------+
|          2|            5|                1|          3|
|          2|           10|                5|          3|
|          1|            5|                1|          5|
|          2|            5|                5|          3|
|          1|           12|                1|          3|
+-----------+-------------+-----------------+-----------+
only showing top 5 rows



In [157]:
dfn.createOrReplaceTempView("dfn")
result = spark.sql("""SELECT apci_groups FROM dfn GROUP BY apci_groups""")
result.show()

+-----------+
|apci_groups|
+-----------+
|          7|
|          3|
|          5|
|          6|
|          1|
|          4|
|          2|
+-----------+



In [158]:
dfn.dtypes

[('sex_of_head', 'string'),
 ('main_language', 'string'),
 ('main_sourceincome', 'string'),
 ('apci_groups', 'string')]

In [159]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
import pandas as pd
import pyspark
import os
import urllib
import sys

from pyspark.sql.functions import *
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.feature import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

In [160]:
features = dfn.columns[:-1]
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(dfn) for column in list(dfn.columns) ]    #list(set(dfn.columns)-set(['label']))


In [193]:
pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(dfn).transform(dfn)

df_r.show()

+-----------+-------------+-----------------+-----------+-----------------+-------------------+-----------------------+-----------------+
|sex_of_head|main_language|main_sourceincome|apci_groups|sex_of_head_index|main_language_index|main_sourceincome_index|apci_groups_index|
+-----------+-------------+-----------------+-----------+-----------------+-------------------+-----------------------+-----------------+
|          2|            5|                1|          3|              0.0|                1.0|                    0.0|              0.0|
|          2|           10|                5|          3|              0.0|                8.0|                    3.0|              0.0|
|          1|            5|                1|          5|              1.0|                1.0|                    0.0|              4.0|
|          2|            5|                5|          3|              0.0|                1.0|                    3.0|              0.0|
|          1|           12|       

In [194]:
df_r = df_r.withColumnRenamed("apci_groups_index", "label")

In [195]:
from pyspark.ml.feature import OneHotEncoder
# One Hot Encoder on indexed features
df_r = OneHotEncoder(inputCol="sex_of_head_index", outputCol="sexVec").transform(df_r)
df_r = OneHotEncoder(inputCol="main_language_index", outputCol="languageVec").transform(df_r)
df_r = OneHotEncoder(inputCol="main_sourceincome_index", outputCol="incsourceVec").transform(df_r)

In [196]:
# Feature assembler as a vector
assembler = VectorAssembler(inputCols = ['sexVec','languageVec','incsourceVec'],outputCol = 'features')
df_r = assembler.transform(df_r)
df_r.show(20)

+-----------+-------------+-----------------+-----------+-----------------+-------------------+-----------------------+-----+-------------+--------------+-------------+--------------------+
|sex_of_head|main_language|main_sourceincome|apci_groups|sex_of_head_index|main_language_index|main_sourceincome_index|label|       sexVec|   languageVec| incsourceVec|            features|
+-----------+-------------+-----------------+-----------+-----------------+-------------------+-----------------------+-----+-------------+--------------+-------------+--------------------+
|          2|            5|                1|          3|              0.0|                1.0|                    0.0|  0.0|(1,[0],[1.0])|(12,[1],[1.0])|(7,[0],[1.0])|(20,[0,2,13],[1.0...|
|          2|           10|                5|          3|              0.0|                8.0|                    3.0|  0.0|(1,[0],[1.0])|(12,[8],[1.0])|(7,[3],[1.0])|(20,[0,9,16],[1.0...|
|          1|            5|                1|     

In [197]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df_r.randomSplit([0.7, 0.3])
